#### 1、 import package

In [3]:
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModelForMaskedLM, DataCollatorForLanguageModeling, TrainingArguments, Trainer

#### 2、load datasets

In [4]:
ds = Dataset.load_from_disk("./wiki_cn_filtered/")
ds

Dataset({
    features: ['source', 'completion'],
    num_rows: 10000
})

In [5]:
ds[0]

{'source': 'wikipedia.zh2307',
 'completion': "西安交通大学博物馆（Xi'an Jiaotong University Museum）是一座位于西安交通大学的博物馆，馆长是锺明善。\n历史\n2004年9月20日开始筹建，2013年4月8日正式建成开馆，位于西安交通大学兴庆校区陕西省西安市咸宁西路28号。建筑面积6,800平米，展厅面积4,500平米，馆藏文物4,900余件。包括历代艺术文物馆、碑石书法馆、西部农民画馆、邢良坤陶瓷艺术馆、陕西秦腔博物馆和书画展厅共五馆一厅。\n营业时间\n* 周一至周六：上午九点至十二点，下午一点至五点\n* 周日闭馆"}

#### 3、Dataset process

In [9]:
tokenizer = AutoTokenizer.from_pretrained("hfl/chinese-macbert-base")

def process_func(examples):
    return tokenizer(examples['completion'], max_length=384, truncation=True)

In [10]:
tokenized_ds = ds.map(process_func, batched=True, remove_columns=ds.column_names)
tokenized_ds

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 10000
})

In [11]:
from torch.utils.data import DataLoader
dl = DataLoader(tokenized_ds, batch_size=2, 
                collate_fn=DataCollatorForLanguageModeling(tokenizer, 
                                                           mlm=True, mlm_probability=0.15))

In [12]:
next(enumerate(dl))

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


(0,
 {'input_ids': tensor([[  101,  6205,  2128,   769,  6858,  1920,  2110,  1300,  4289,  7667,
            103, 13135,   112,  9064, 12095,  8731,  8626,   103,  8736, 10553,
           8021,  3221,   671,  2429,   855,   754,  6205,  2128,   769,  6858,
           1920,   103,  4638,   103,   103,  7667,  8024,  7667,  7270,   103,
           7247,  3209,  1587,   511,  1325,  1380,  8258,  2399,   130,  3299,
           8113,  3189,  2458,  1993,  5040,  2456, 18781,   103,  2399,   125,
           3299,   129,  3189,  3633,  2466,   103,  2768,  2458,  7667,  8024,
            855,   754,  6205,  2128,   103,  6858,  1920,  2110,  1069,  2412,
           3413,   103,  7362,  6205,  4689,  6205,  2128, 18398,  1496,  2123,
           6205,  6662,  8143,   103,   511,  2456,  5029,  7481,  4916,   127,
            117,   103,  2398,  5101,  8024,  2245,  1324,  7481,  4916,   125,
            117,  8195,  2398,  5101,  8024,  7667,  5966,  3152,  4289,   125,
            103,  8567

In [13]:
tokenizer.mask_token, tokenizer.mask_token_id

('[MASK]', 103)

#### 4、create model

In [14]:
model = AutoModelForMaskedLM.from_pretrained("hfl/chinese-macbert-base")

Some weights of the model checkpoint at hfl/chinese-macbert-base were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


#### 5、Configure Train Parameters

In [17]:
args = TrainingArguments(
    output_dir="./masked_lm", 
    per_device_train_batch_size=32, 
    logging_steps=10, 
    num_train_epochs=1
    )

#### 6、Configure Trainer

In [18]:
trainer = Trainer(
    args=args, 
    model=model, 
    train_dataset=tokenized_ds, 
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=True, mlm_probability=0.15)
)

#### 7、 model train

In [19]:
trainer.train()

/root/miniconda3/envs/torch/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss
10,1.356900
20,1.333300
30,1.336400
40,1.300500
50,1.328800
60,1.323400
70,1.290000


TrainOutput(global_step=79, training_loss=1.3197419251067728, metrics={'train_runtime': 75.1889, 'train_samples_per_second': 132.998, 'train_steps_per_second': 1.051, 'total_flos': 1973819658240000.0, 'train_loss': 1.3197419251067728, 'epoch': 1.0})

#### 8、model infer

In [21]:
from transformers import pipeline
pipe = pipeline("fill-mask", model=model, tokenizer=tokenizer, device=0)

In [23]:
res = pipe("下面是一则[MASK][MASK]新闻。小编报道，近日，游戏产业发展的非常")
res

[[{'score': 0.14859214425086975,
   'token': 7028,
   'token_str': '重',
   'sequence': '[CLS] 下 面 是 一 则 重 [MASK] 新 闻 。 小 编 报 道 ， 近 日 ， 游 戏 产 业 发 展 的 非 常 [SEP]'},
  {'score': 0.07158365100622177,
   'token': 2031,
   'token_str': '娱',
   'sequence': '[CLS] 下 面 是 一 则 娱 [MASK] 新 闻 。 小 编 报 道 ， 近 日 ， 游 戏 产 业 发 展 的 非 常 [SEP]'},
  {'score': 0.05919073894619942,
   'token': 3297,
   'token_str': '最',
   'sequence': '[CLS] 下 面 是 一 则 最 [MASK] 新 闻 。 小 编 报 道 ， 近 日 ， 游 戏 产 业 发 展 的 非 常 [SEP]'},
  {'score': 0.0586271733045578,
   'token': 4178,
   'token_str': '热',
   'sequence': '[CLS] 下 面 是 一 则 热 [MASK] 新 闻 。 小 编 报 道 ， 近 日 ， 游 戏 产 业 发 展 的 非 常 [SEP]'},
  {'score': 0.0577559620141983,
   'token': 3173,
   'token_str': '新',
   'sequence': '[CLS] 下 面 是 一 则 新 [MASK] 新 闻 。 小 编 报 道 ， 近 日 ， 游 戏 产 业 发 展 的 非 常 [SEP]'}],
 [{'score': 0.1811920702457428,
   'token': 6206,
   'token_str': '要',
   'sequence': '[CLS] 下 面 是 一 则 [MASK] 要 新 闻 。 小 编 报 道 ， 近 日 ， 游 戏 产 业 发 展 的 非 常 [SEP]'},
  {'score': 0.0958498269319534